In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from numpy import log
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf
from datetime import datetime, timedelta

In [3]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df1 = pd.read_excel('biometria.xlsx', sheet_name='Página1')
print(df1.head(10))

   Index   peso  comprimento_Total  comprimento_Padrao comprimento_Cabeça  \
0      1  51.45               14.3                12.2                  4   
1      2  37.15               12.4                10.9                3.8   
2      3  47.26               13.7                11.2                3.4   
3      4  36.67               12.9                11.1                3.7   
4      5  45.48               13.2                11.3                3.8   
5      6  36.77               12.5                10.8                3.2   
6      7  32.79               12.0                10.2                3.4   
7      8  35.52               12.4                10.8                3.4   
8      9  35.58               12.3                11.0                3.4   
9     10  42.88               13.2                11.2                3.8   

   altura  espessura       data  
0     4.1        1.9 2024-02-28  
1     3.2        1.2 2024-02-28  
2     4.0        1.7 2024-02-28  
3     3.7       

In [5]:
df1 = df1.drop(columns=[
'data'

])
df1.head()

KeyError: "['data'] not found in axis"

In [1]:
#from pycaret.datasets import get_data
#boston = get_data('boston')
from pycaret.regression import *
exp_name = setup(data = df1,  target = 'peso')
best_model = compare_models()

NameError: name 'df1' is not defined